В качестве домашнего задания вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить 4 модели которые будут предсказывать целевую переменную <b>RainTomorrow</b> с помощью:

   1. логистической регрессии [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
   
   2. метода ближайших соседей [sklearn.neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
 
   3. Байесовского классификатора [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
   
   4. логистической регрессии реализованной самостоятельно

Затем следует сравнить результаты моделей (по качеству и времени выполнения) и сделать вывод о том, какая модель и с какими параметрами даёт лучшие результаты.

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

In [1]:
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import BayesianRidge
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

%matplotlib notebook

### Реализация логистической регрессии
__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [2]:
theta = np.array([1, 2, 3])

X =  np.array([[ 1,  1, 1],
               [-1, -2, 1],
               [-1, -2, 2],
               [-2, -2, -3]
              ])

y = np.array([1, 1, 0, 0])

In [3]:
def probability(theta, X):
    d = -np.dot(X, theta)
    ex = np.exp(d)
    result = 1 / (1 + ex)
    return result

prob = probability(theta, X)

assert type(prob) == np.ndarray, 'Возвращается неверный тип'
assert prob.shape == (X.shape[0],), 'Неверный размер массива'
assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [4]:
def binary_class_prediction(theta, X, threshold =.5):
    prob =  probability(theta, X)
    result = np.where(prob > 0.5, 1, 0)
    # YOUR CODE HERE
    return result

y_pred = binary_class_prediction(theta, X)


assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'
assert min(y_pred) == 0, 'Функция считается неверно'
assert max(y_pred) == 1, 'Функция считается неверно'

__Функционал качества логистической регрессии__

Запишем правдободовие выборки для меток класса $y \in \{+1, -1\}$ 

$$Likelihood(a, X^\ell) = \prod_{i = 1}^{\ell} a(x_i,\theta)^{[y_i = +1]} (1 - a(x_i, \theta))^{[y_i = -1]} → \operatorname*{max}_{\theta}$$ 

Прологарифмируем правдоподобие выборки и перейдем к задаче минимизации:

$$Q(a, X^\ell) =     -\sum_{i = 1}^{\ell} 
        [y_i = +1] \log a(x_i, \theta)
        +
        [y_i = -1] \log (1 - a(x_i, \theta)) \to \operatorname*{min}_{\theta}$$ 
        
Подставим $a(x, \theta)$ в функцинал качества:

$$ Q(a, X^\ell) = -\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{\exp(-\langle \theta, x_i \rangle)}{1 + \exp(-\langle \theta, x_i \rangle)}
\right)
=\\
=
-\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{1}{1 + \exp(\langle \theta, x_i \rangle)}
\right)
=\\
=
\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) $$
    

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:

In [5]:
def logloss(theta, X, y):
    l = len(X)
    y_inner = y.copy()
    y_inner[y==0] = -1
    summation = np.sum(np.log(1 + np.exp(-y_inner * np.dot(X, theta))))
    return (1 / l) * summation

In [6]:
assert logloss(theta, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

<b>Вход: </b> Выборка $X^\ell$, темп обучения $h$

<b>Выход: </b> оптимальный вектор весов $\theta$

1.  Инициализировать веса $\theta$
2.  Инициализировать оценку функционала качества: $Q(a, X^\ell)$
3.  <b>Повторять</b>: 

    Выбрать случайным образом подвыборку объектов $X^{batch} =\{x_1, \dots,x_n \}$ из $X^{\ell}$
    
    Рассчитать градиент функционала качества: $\nabla Q(X^{batch}, \theta)$
    
    Обновить веса: $\theta := \theta - h\cdot \nabla Q(X^{batch}, \theta)$
       
    <b>Пока</b> значение $Q$ и/или веса $\theta$ не сойдутся   

Реализуем функцию рассчета градиента функционала качества

$$\frac{\partial Q(a, X^{batch}) }{\partial \theta_j}   = \frac{\partial \frac{1}{n}\sum_{i = 1}^{n}
    \log \left(
        1 + \exp(- y_i \langle \theta, x_i \rangle)
    \right)} {\partial \theta_j}  = \frac{1}{n}\sum_{i = 1}^{n}
     \frac {1}{
        1 + \exp(- y_i \langle \theta, x_i \rangle)} \cdot  \exp(- y_i \langle \theta, x_i \rangle) \cdot -y_i x_{ij}$$

Реализуйте рассчет градиента в матричном виде:

In [7]:
def gradient(theta, X, y):
    y_new = y.copy()
    y_new[y==0] = -1
    n = len(X)

    summ = np.exp(-y_new * np.dot(X, theta)) / (1 + np.exp(-y_new * np.dot(X, theta)))
    
    result = (1/n) * np.dot(X.T, summ * (-y_new))
    return result

assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'

Функция обучения уже реализована

In [8]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)
    
    errors = []
    
    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
    # plt 
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')
        
        
    for _ in range(iters):  
        
        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        # считаем производные
        grad = gradient(theta, X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'
        
        
        # Обновляем веса
        
        theta -= grad * h
        
        theta_history = np.vstack((theta_history, theta))
        
        # error
        loss = logloss(theta, X, y)
        errors.append(loss)
        
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')
            
            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()   
            
    return theta

In [9]:
X, y = make_classification(n_samples=2000)

In [10]:
optimal_theta = fit(X, y)

<IPython.core.display.Javascript object>

In [11]:
y_pred = binary_class_prediction(optimal_theta, X)

In [12]:
y_pred

array([1, 0, 1, ..., 0, 1, 0])

### Рассмотрим датасет

In [13]:
X = pd.read_csv('weather.csv')
X

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,145454,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,...,59.0,27.0,1024.7,1021.2,NaN,NaN,9.4,20.9,No,No
142189,145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
142190,145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
142191,145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No


In [14]:
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})

In [15]:
del X['RainTomorrow']

In [16]:
print(X)

        Unnamed: 0        Date Location  MinTemp  MaxTemp  Rainfall  \
0                0  2008-12-01   Albury     13.4     22.9       0.6   
1                1  2008-12-02   Albury      7.4     25.1       0.0   
2                2  2008-12-03   Albury     12.9     25.7       0.0   
3                3  2008-12-04   Albury      9.2     28.0       0.0   
4                4  2008-12-05   Albury     17.5     32.3       1.0   
...            ...         ...      ...      ...      ...       ...   
142188      145454  2017-06-20    Uluru      3.5     21.8       0.0   
142189      145455  2017-06-21    Uluru      2.8     23.4       0.0   
142190      145456  2017-06-22    Uluru      3.6     25.3       0.0   
142191      145457  2017-06-23    Uluru      5.4     26.9       0.0   
142192      145458  2017-06-24    Uluru      7.8     27.0       0.0   

        Evaporation  Sunshine WindGustDir  WindGustSpeed  ... WindSpeed3pm  \
0               NaN       NaN           W           44.0  ...        

In [17]:
set(X['WindDir9am'].unique()).union(set(X['WindDir3pm'].unique()))

{'E',
 'ENE',
 'ESE',
 'N',
 'NE',
 'NNE',
 'NNW',
 'NW',
 'S',
 'SE',
 'SSE',
 'SSW',
 'SW',
 'W',
 'WNW',
 'WSW',
 nan}

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     142193 non-null  int64  
 1   Date           142193 non-null  object 
 2   Location       142193 non-null  object 
 3   MinTemp        141556 non-null  float64
 4   MaxTemp        141871 non-null  float64
 5   Rainfall       140787 non-null  float64
 6   Evaporation    81350 non-null   float64
 7   Sunshine       74377 non-null   float64
 8   WindGustDir    132863 non-null  object 
 9   WindGustSpeed  132923 non-null  float64
 10  WindDir9am     132180 non-null  object 
 11  WindDir3pm     138415 non-null  object 
 12  WindSpeed9am   140845 non-null  float64
 13  WindSpeed3pm   139563 non-null  float64
 14  Humidity9am    140419 non-null  float64
 15  Humidity3pm    138583 non-null  float64
 16  Pressure9am    128179 non-null  float64
 17  Pressure3pm    128212 non-nul

In [19]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


def transform_dataframe(df):
    # Удаление колонки 'Unnamed: 0'
    df = df.drop(columns=['Unnamed: 0'])

    # Кодирование колонки 'Date' порядковым числом
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].map(pd.Timestamp.toordinal)

    # # One Hot Encoding для колонки 'Location'
    # encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    # location_encoded = encoder.fit_transform(df[['Location']]) if fit else encoder.transform(df[['Location']])
    # location_df = pd.DataFrame(location_encoded, columns=encoder.get_feature_names_out(['Location']))
    # df = pd.concat([df.drop(columns=['Location']), location_df], axis=1)

    # Заполнение пропусков для числовых колонок с помощью Regression Imputation
    numerical_cols = df.select_dtypes(include=['float64']).columns
    print("Заполнение пропусков для числовых колонок")
    imputer = IterativeImputer(estimator=BayesianRidge(), random_state=0)
    df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

    # Заполнение пропусков для категориальных колонок с сохранением распределения
    categorical_cols = df.select_dtypes(include=['object']).columns
    print("Заполнение пропусков для категориальных колонок")
    for col in tqdm(categorical_cols, desc="Категориальные колонки"):
        df[col] = df[col].apply(lambda x: np.random.choice(df[col].dropna()) if pd.isna(x) else x)

    # Кодирование направлений ветра через sin и cos
    def encode_wind_direction(direction):
        directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
        angle = directions.index(direction) * (360 / len(directions))
        return np.sin(np.deg2rad(angle)), np.cos(np.deg2rad(angle))

    wind_dir_cols = ['WindDir9am', 'WindDir3pm', 'WindGustDir']
    print("Кодирование направлений ветра через sin и cos")
    for col in tqdm(wind_dir_cols, desc="Направления ветра"):
        sin_col = col + '_sin'
        cos_col = col + '_cos'
        df[[sin_col, cos_col]] = df[col].apply(lambda x: pd.Series(encode_wind_direction(x)))

    # Удаление исходных колонок направлений ветра
    df = df.drop(columns=wind_dir_cols)

    # Преобразование 'RainToday' в 1 и 0
    df.RainToday.replace({'No':0, 'Yes': 1}, inplace=True)

    return df

In [20]:
from sklearn.preprocessing import MinMaxScaler

def normalize_dataframe(df):
    # Создание копии датафрейма, чтобы не изменять оригинальный датафрейм
    df_normalized = df.copy()

    # Определение колонок с числовыми данными
    numerical_cols = df_normalized.select_dtypes(include=['float64', 'int64']).columns

    scaler = MinMaxScaler()

    # Применение MinMaxScaler к числовым колонкам
    df_normalized[numerical_cols] = scaler.fit_transform(df_normalized[numerical_cols])

    return df_normalized

In [21]:
def encode_location(train_df, test_df):
    # Объединение обучающего и тестового наборов данных для обучения OneHotEncoder
    combined_df = pd.concat([train_df, test_df], ignore_index=True)

    # Применение One Hot Encoding к колонке 'Location'
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    location_encoded = encoder.fit_transform(combined_df[['Location']])

    location_df = pd.DataFrame(location_encoded, columns=encoder.get_feature_names_out(['Location']))

    combined_df = pd.concat([combined_df.drop(columns=['Location']), location_df], axis=1)

    # Разделение обратно на обучающий и тестовый наборы данных
    train_df_transformed = combined_df.iloc[:len(train_df), :].reset_index(drop=True)
    test_df_transformed = combined_df.iloc[len(train_df):, :].reset_index(drop=True)

    return train_df_transformed, test_df_transformed

In [22]:
def prepare_data(X_train, X_test):
    X_train = transform_dataframe(X_train)
    X_test = transform_dataframe(X_test)
    X_train, X_test = encode_location(X_train, X_test)
    X_train = normalize_dataframe(X_train) 
    X_test = normalize_dataframe(X_test)
    return X_train, X_test

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

In [24]:
X_train['Location'].unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany'], dtype=object)

In [25]:
X_test['Location'].unique()

array(['Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport', 'Perth',
       'SalmonGums', 'Walpole', 'Hobart', 'Launceston', 'AliceSprings',
       'Darwin', 'Katherine', 'Uluru'], dtype=object)

In [26]:
X_train, X_test = prepare_data(X_train, X_test)

Заполнение пропусков для числовых колонок
Заполнение пропусков для категориальных колонок


Категориальные колонки: 100%|██████████| 5/5 [00:32<00:00,  6.42s/it]


Кодирование направлений ветра через sin и cos


Направления ветра: 100%|██████████| 3/3 [00:18<00:00,  6.16s/it]


Заполнение пропусков для числовых колонок
Заполнение пропусков для категориальных колонок


Категориальные колонки: 100%|██████████| 5/5 [00:03<00:00,  1.41it/s]


Кодирование направлений ветра через sin и cos


Направления ветра: 100%|██████████| 3/3 [00:06<00:00,  2.14s/it]


In [27]:
X_train

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,Location_Townsville,Location_Tuggeranong,Location_Uluru,Location_WaggaWagga,Location_Walpole,Location_Watsonia,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera
0,0.112372,0.534938,0.523629,0.018415,0.076646,0.410703,0.276327,0.153846,0.275862,0.774694,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.112656,0.398822,0.565217,0.016825,0.074867,0.567663,0.276327,0.030769,0.252874,0.631326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.112940,0.523595,0.576560,0.016825,0.090781,0.605667,0.291264,0.146154,0.298851,0.599467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.113224,0.439657,0.620038,0.016825,0.079233,0.597161,0.126961,0.084615,0.103448,0.636636,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.113507,0.627950,0.701323,0.019475,0.084987,0.348418,0.253922,0.053846,0.229885,0.833104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106639,0.302781,0.462343,0.432892,0.016825,0.061441,0.463987,0.264299,0.069231,0.356322,0.769385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106640,0.303065,0.491835,0.398866,0.037495,0.049443,0.176806,0.170320,0.053846,0.149425,0.870273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106641,0.303348,0.514520,0.427221,0.016825,0.048110,0.257216,0.426494,0.153846,0.528736,0.764075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106642,0.303632,0.514520,0.396975,0.018415,0.053442,0.536740,0.104758,0.030769,0.149425,0.657876,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
X_test

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,Location_Townsville,Location_Tuggeranong,Location_Uluru,Location_WaggaWagga,Location_Walpole,Location_Watsonia,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera
0,0.252667,0.412088,0.541985,0.015400,0.173306,0.663704,0.243733,0.107135,0.292308,0.721176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.252972,0.450549,0.519084,0.015400,0.165790,0.663704,0.387700,0.107135,0.569231,0.671840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.253276,0.513736,0.274809,0.017007,0.188337,0.248876,0.407455,0.076868,0.569231,0.737621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.253581,0.359890,0.203562,0.070576,0.113183,0.480982,0.274549,0.303868,0.261538,0.655395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.253886,0.329670,0.264631,0.015400,0.116940,0.639012,0.151051,0.122268,0.230769,0.630727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35544,0.998476,0.239011,0.376590,0.015400,0.132853,0.590312,0.201613,0.243335,0.200000,0.647172,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35545,0.998781,0.219780,0.417303,0.015400,0.145708,0.612535,0.201613,0.213068,0.169231,0.581391,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35546,0.999086,0.241758,0.465649,0.015400,0.143781,0.643015,0.129032,0.213068,0.138462,0.622504,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35547,0.999390,0.291209,0.506361,0.015400,0.158764,0.610782,0.250000,0.152535,0.138462,0.597836,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
class CustomLogisticRegression:
    def __init__(self, batch_size=10, h=0.05, iters=100, plot=False):
        self.batch_size = batch_size
        self.h = h
        self.iters = iters
        self.plot = plot
        self.theta = None
    
    def fit(self, X, y):

        # получаем размерности матрицы
        size, dim = X.shape
    
        # случайная начальная инициализация
        theta = np.random.uniform(size=dim)
    
        errors = []
    
        theta_history = theta
        colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
        # plt 
        if self.plot:
            fig = plt.figure(figsize=(15, 10))
            ax1 = fig.add_subplot(221)
            ax2 = fig.add_subplot(222)
            ax3 = fig.add_subplot(212)
            fig.suptitle('Gradient descent')
    
    
        for _ in range(self.iters):
    
            # берём случайный набор элементов
            batch = np.random.choice(size, self.batch_size, replace=False)
            X_batch = X.iloc[batch]
            y_batch = y.iloc[batch]
    
            # считаем производные
            grad = gradient(theta, X_batch, y_batch)
    
            assert type(grad) == np.ndarray, 'неверный тип'
            assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
            assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'
    
    
            # Обновляем веса
    
            theta -= grad * self.h
    
            theta_history = np.vstack((theta_history, theta))
    
            # error
            loss = logloss(theta, X, y)
            errors.append(loss)
    
            if self.plot:
                ax1.clear()
                ax1.scatter(range(dim), theta, label='Gradient solution')
                ax1.legend(loc="upper left")
                ax1.set_title('theta')
                ax1.set_ylabel(r'$\bar \beta$')
                ax1.set_xlabel('weight ID')
    
    
                ax2.plot(range(_+1), errors, 'g-')
                ax2.set_title('logloss')
                ax2.set_xlabel('itarations')
    
                ax3.plot(theta_history)
                ax3.set_title('update theta')
                ax3.set_ylabel('value')
                ax3.set_xlabel('itarations')
                time.sleep(0.05)
                fig.canvas.draw()
                
            self.theta = theta
    
    def predict(self, X, threshold =.5):
        prob =  probability(self.theta, X)
        result = np.where(prob > 0.5, 1, 0)
        # YOUR CODE HERE
        return result

MyClassifier = CustomLogisticRegression()

In [30]:
def compare_classifiers(X_train, X_test, y_train, y_test):
    classifiers = {
        "Logistic Regression": LogisticRegression(max_iter=10000),
        "K-Nearest Neighbors": KNeighborsClassifier(),
        "Naive Bayes": GaussianNB(),
        "Custom Logistic Regression": CustomLogisticRegression()
    }

    f1_scores = []
    times = []

    for name, clf in classifiers.items():
        start_time = time.time()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        end_time = time.time()

        f1 = f1_score(y_test, y_pred)
        elapsed_time = end_time - start_time

        f1_scores.append(f1)
        times.append(elapsed_time)
        print(f"{name}: F1-Score = {f1:.4f}, Time = {elapsed_time:.4f} seconds")

    # Построение столбчатой диаграммы для F1-Score
    fig, ax = plt.subplots(2, 1, figsize=(10, 10))
    index = np.arange(len(classifiers))
    bar_width = 0.35

    ax[0].bar(index, f1_scores, bar_width, label='F1-Score', color='b')
    ax[0].set_xlabel('Classifiers')
    ax[0].set_ylabel('F1-Score')
    ax[0].set_title('Comparison of Classifiers - F1-Score')
    ax[0].set_xticks(index)
    ax[0].set_xticklabels(classifiers.keys(), rotation=45)
    ax[0].legend()

    # Построение столбчатой диаграммы для времени выполнения
    ax[1].bar(index, times, bar_width, label='Time (s)', color='g')
    ax[1].set_xlabel('Classifiers')
    ax[1].set_ylabel('Time (s)')
    ax[1].set_title('Comparison of Classifiers - Time')
    ax[1].set_xticks(index)
    ax[1].set_xticklabels(classifiers.keys(), rotation=45)
    ax[1].legend()

    plt.tight_layout()
    plt.show()

In [31]:
compare_classifiers(X_train, X_test, Y_train, Y_test)

Logistic Regression: F1-Score = 0.4100, Time = 3.6676 seconds
K-Nearest Neighbors: F1-Score = 0.4731, Time = 1.6022 seconds
Naive Bayes: F1-Score = 0.5738, Time = 0.0985 seconds
Custom Logistic Regression: F1-Score = 0.1948, Time = 2.3299 seconds


<IPython.core.display.Javascript object>

In [34]:
def compare_classifiers(X_train, X_test, y_train, y_test):
    classifiers = {
        "Logistic Regression": LogisticRegression(max_iter=10000),
        "K-Nearest Neighbors": KNeighborsClassifier(),
        "Naive Bayes": GaussianNB(),
        "Custom Logistic Regression": CustomLogisticRegression(batch_size=40, iters=10000)
    }

    f1_scores = []
    times = []

    for name, clf in classifiers.items():
        start_time = time.time()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        end_time = time.time()

        f1 = f1_score(y_test, y_pred)
        elapsed_time = end_time - start_time

        f1_scores.append(f1)
        times.append(elapsed_time)
        print(f"{name}: F1-Score = {f1:.4f}, Time = {elapsed_time:.4f} seconds")

    # Построение столбчатой диаграммы для F1-Score
    fig, ax = plt.subplots(2, 1, figsize=(10, 10))
    index = np.arange(len(classifiers))
    bar_width = 0.35

    ax[0].bar(index, f1_scores, bar_width, label='F1-Score', color='b')
    ax[0].set_xlabel('Classifiers')
    ax[0].set_ylabel('F1-Score')
    ax[0].set_title('Comparison of Classifiers - F1-Score')
    ax[0].set_xticks(index)
    ax[0].set_xticklabels(classifiers.keys(), rotation=45)
    ax[0].legend()

    # Построение столбчатой диаграммы для времени выполнения
    ax[1].bar(index, times, bar_width, label='Time (s)', color='g')
    ax[1].set_xlabel('Classifiers')
    ax[1].set_ylabel('Time (s)')
    ax[1].set_title('Comparison of Classifiers - Time')
    ax[1].set_xticks(index)
    ax[1].set_xticklabels(classifiers.keys(), rotation=45)
    ax[1].legend()

    plt.tight_layout()
    plt.show()

In [35]:
compare_classifiers(X_train, X_test, Y_train, Y_test)

Logistic Regression: F1-Score = 0.4100, Time = 3.7843 seconds
K-Nearest Neighbors: F1-Score = 0.4731, Time = 1.7084 seconds
Naive Bayes: F1-Score = 0.5738, Time = 0.1225 seconds
Custom Logistic Regression: F1-Score = 0.5641, Time = 246.7765 seconds


<IPython.core.display.Javascript object>

Как видно из графиков Байесовский классификатор справляется лучше и быстрее всего. Мне очень хотелось, чтобы моя самописная регрессия "выиграла" хотябы у одного из встроенных алгоритмов классификации sklearn, поэтому я увеличил количество итераций и размер батча. При таком раскладе моя регрессия приблизилась по качеству к Байесовскому классификатору, однако требует при этом неприличное количество времени (246 секунд на моей машине против 0.1225) но я всё равно доволен)